![spotify_logo](../img/spotify_logo.png)

# Spotify REST API Challenge

__What to listen?__

Create your own playlist based on the related artists and their most popular tracks using the [Spotify REST API](https://developer.spotify.com/)

In [33]:
# Imports
import pandas as pd
import requests
import numpy as np

### Get access!!!

Get your `client_id` and `client_secret` to generate your __token__ access

In [34]:
# Generate token with a POST request

client_id = # CLIENT ID
client_secret = # CLIENT SECRET
auth_url = 'https://accounts.spotify.com/api/token'

In [35]:
auth_response = requests.post(auth_url, {'grant_type': 'client_credentials',
                                         'client_id': client_id,
                                         'client_secret': client_secret}).json()

In [1]:
access_token = auth_response['access_token']
auth_response

NameError: name 'auth_response' is not defined

### Set your main variables!!!

Set the `base_uri` (i.e.: end-point), parameters and `headers` for your __GET__ operations

In [37]:
# Base end-point construction

base_url = 'https://api.spotify.com/v1/'
header_info = {'Authorization': 'Bearer {token}'.format(token=access_token)}

In [2]:
header_info

NameError: name 'header_info' is not defined

### Create your new playlist!!!

Use [`/related-artists`](https://developer.spotify.com/documentation/web-api/reference/#/operations/get-an-artists-related-artists) and [`/top-tracks`](https://developer.spotify.com/documentation/web-api/reference/#/operations/get-an-artists-top-tracks) in order to get the data that you need.

__Here you have the different artists to start with:__

- __Option 1:__ 0L8ExT028jH3ddEcZwqJJ5 `"Sublime!"`

- __Option 2:__ 4Y7tXHSEejGu1vQ9bwDdXW `"Sublime!"`

- __Option 3:__ 6FBDaR13swtiWwGhX1WQsP `"Mmmmm...ok!"`

- __Option 4:__ 0kyQwKHCZnKE7kTXkxXjrB `"Not my faves!"`

- __Option 5:__ 2d0hyoQ5ynDBnkvAbJKORj `"Sublime!"`

- __Option 6:__ 3bgsNtcf5d5h9jbQbohfBK `"Not my faves!"`

- __Option 7:__ 7mWCSSOYqm4E9mB7V4ot6S `"Not my faves!"`

- __Option 8:__ 64KEffDW9EtZ1y2vBYgq8T `"Moody ones!"`

- __Option 9:__ 4k1ELeJKT1ISyDv8JivPpB `"Some serious shit!"`

- __Option 10:__ 4Z8W4fKeB5YxbusRsdQVPb `"Some serious shit!"`

- __Option 11:__ 26dSoYclwsYLMAKD3tpOr4 `"It fucking sucks as shit!"`

- __Option 12:__ 7y97mc3bZRFXzT2szRM4L4 `Sip of wine and "hohohohoh!" (Intellectual laugh)`

- __Option 13:__ 1w5Kfo2jwwIPruYS2UWh56 `"Sublime!"`


> Remember to check the [Requests](https://requests.readthedocs.io/en/latest/) library docs!!!

---

# Borrador

### 1. Artist chosen (Option 1):

#### Red Hot Chili Peppers - Give it away

![give_it_away](https://i.gifer.com/No0a.gif)

In [39]:
print(base_url)

https://api.spotify.com/v1/


In [22]:
artist_id = '0L8ExT028jH3ddEcZwqJJ5'
resource = 'artists/'

# Artist endpoint
artist_ep = base_url + resource + artist_id
print(artist_ep)

https://api.spotify.com/v1/artists/0L8ExT028jH3ddEcZwqJJ5


In [23]:
# API call
artist_response = requests.get(artist_ep, headers=header_info)
artist_response.status_code

200

In [24]:
artist_response = artist_response.json()
artist_response

{'external_urls': {'spotify': 'https://open.spotify.com/artist/0L8ExT028jH3ddEcZwqJJ5'},
 'followers': {'href': None, 'total': 20275607},
 'genres': ['alternative rock',
  'funk metal',
  'funk rock',
  'permanent wave',
  'rock'],
 'href': 'https://api.spotify.com/v1/artists/0L8ExT028jH3ddEcZwqJJ5',
 'id': '0L8ExT028jH3ddEcZwqJJ5',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab6761610000e5ebc33cc15260b767ddec982ce8',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/ab67616100005174c33cc15260b767ddec982ce8',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/ab6761610000f178c33cc15260b767ddec982ce8',
   'width': 160}],
 'name': 'Red Hot Chili Peppers',
 'popularity': 80,
 'type': 'artist',
 'uri': 'spotify:artist:0L8ExT028jH3ddEcZwqJJ5'}

### 2. Artist top tracks

In [25]:
# Getting the chosen artist top tracks
artist_data = {
    "id": artist_response["id"],
    "href": artist_response["external_urls"]["spotify"],
    "name": artist_response["name"],
    "uri": artist_response["uri"],
    "artist_name": artist_response["name"]
}
artist_df = pd.DataFrame([artist_data])

# Display artist DataFrame
print("Artist DataFrame:")
artist_df

Artist DataFrame:


,id,href,name,uri,artist_name
0,0L8ExT028jH3ddEcZwqJJ5,https://open.spotify.com/artist/0L8ExT028jH3dd...,Red Hot Chili Peppers,spotify:artist:0L8ExT028jH3ddEcZwqJJ5,Red Hot Chili Peppers


In [29]:
parameter = '/top-tracks'
country_market = '?market=ES'   
top_tracks_ep = artist_ep + parameter + country_market    
top_tracks_ep_response = requests.get(top_tracks_ep, headers=header_info).json()
top_tracks_ep_response

{'tracks': [{'album': {'album_type': 'album',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/0L8ExT028jH3ddEcZwqJJ5'},
      'href': 'https://api.spotify.com/v1/artists/0L8ExT028jH3ddEcZwqJJ5',
      'id': '0L8ExT028jH3ddEcZwqJJ5',
      'name': 'Red Hot Chili Peppers',
      'type': 'artist',
      'uri': 'spotify:artist:0L8ExT028jH3ddEcZwqJJ5'}],
    'external_urls': {'spotify': 'https://open.spotify.com/album/30Perjew8HyGkdSmqguYyg'},
    'href': 'https://api.spotify.com/v1/albums/30Perjew8HyGkdSmqguYyg',
    'id': '30Perjew8HyGkdSmqguYyg',
    'images': [{'height': 640,
      'url': 'https://i.scdn.co/image/ab67616d0000b273153d79816d853f2694b2cc70',
      'width': 640},
     {'height': 300,
      'url': 'https://i.scdn.co/image/ab67616d00001e02153d79816d853f2694b2cc70',
      'width': 300},
     {'height': 64,
      'url': 'https://i.scdn.co/image/ab67616d00004851153d79816d853f2694b2cc70',
      'width': 64}],
    'is_playable': True,
    'name': 'Bl

In [30]:
def get_top_tracks(artist_id):    
    # Extracting relevant information for each track
    tracks_data = []
    for track in top_tracks_ep_response["tracks"]:
        track_data = {
            "id": track["id"],
            "href": track["href"],
            "song_name": track["name"],
            "uri": track["uri"],
            "artist_name": artist_response["name"]
        }
        tracks_data.append(track_data)
    
    # Creating the DataFrame
    top_tracks_df = pd.DataFrame(tracks_data)
    
    return top_tracks_df

# Get and display top tracks DataFrame
artist_top_tracks = get_top_tracks(artist_id)
print("\nTop Tracks DataFrame:")
artist_top_tracks


Top Tracks DataFrame:


,id,href,song_name,uri,artist_name
0,3d9DChrdc6BOeFsbrZ3Is0,https://api.spotify.com/v1/tracks/3d9DChrdc6BO...,Under the Bridge,spotify:track:3d9DChrdc6BOeFsbrZ3Is0,Red Hot Chili Peppers
1,48UPSzbZjgc449aqz8bxox,https://api.spotify.com/v1/tracks/48UPSzbZjgc4...,Californication,spotify:track:48UPSzbZjgc449aqz8bxox,Red Hot Chili Peppers
2,3ZOEytgrvLwQaqXreDs2Jx,https://api.spotify.com/v1/tracks/3ZOEytgrvLwQ...,Can't Stop,spotify:track:3ZOEytgrvLwQaqXreDs2Jx,Red Hot Chili Peppers
3,1G391cbiT3v3Cywg8T7DM1,https://api.spotify.com/v1/tracks/1G391cbiT3v3...,Scar Tissue,spotify:track:1G391cbiT3v3Cywg8T7DM1,Red Hot Chili Peppers
4,64BbK9SFKH2jk86U3dGj2P,https://api.spotify.com/v1/tracks/64BbK9SFKH2j...,Otherside,spotify:track:64BbK9SFKH2jk86U3dGj2P,Red Hot Chili Peppers
5,2aibwv5hGXSgw7Yru8IYTO,https://api.spotify.com/v1/tracks/2aibwv5hGXSg...,Snow (Hey Oh),spotify:track:2aibwv5hGXSgw7Yru8IYTO,Red Hot Chili Peppers
6,10Nmj3JCNoMeBQ87uw5j8k,https://api.spotify.com/v1/tracks/10Nmj3JCNoMe...,Dani California,spotify:track:10Nmj3JCNoMeBQ87uw5j8k,Red Hot Chili Peppers
7,1f2V8U1BiWaC9aJWmpOARe,https://api.spotify.com/v1/tracks/1f2V8U1BiWaC...,By the Way,spotify:track:1f2V8U1BiWaC9aJWmpOARe,Red Hot Chili Peppers
8,0uppYCG86ajpV2hSR3dJJ0,https://api.spotify.com/v1/tracks/0uppYCG86ajp...,Give It Away,spotify:track:0uppYCG86ajpV2hSR3dJJ0,Red Hot Chili Peppers
9,2oaK4JLVnmRGIO9ytBE1bt,https://api.spotify.com/v1/tracks/2oaK4JLVnmRG...,Dark Necessities,spotify:track:2oaK4JLVnmRGIO9ytBE1bt,Red Hot Chili Peppers


In [31]:
method = '/related-artists'
rel_artists_ep = base_url + resource + artist_id + method
rel_artists_ep_response = requests.get(rel_artists_ep, headers=header_info).json()
rel_artists_ep_response

{'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7jy3rLJdDQY21OgRLCZ9sD'},
   'followers': {'href': None, 'total': 11382646},
   'genres': ['alternative metal',
    'alternative rock',
    'modern rock',
    'permanent wave',
    'post-grunge',
    'rock'],
   'href': 'https://api.spotify.com/v1/artists/7jy3rLJdDQY21OgRLCZ9sD',
   'id': '7jy3rLJdDQY21OgRLCZ9sD',
   'images': [{'height': 640,
     'url': 'https://i.scdn.co/image/ab6761610000e5ebc884df599abc793c116cdf15',
     'width': 640},
    {'height': 320,
     'url': 'https://i.scdn.co/image/ab67616100005174c884df599abc793c116cdf15',
     'width': 320},
    {'height': 160,
     'url': 'https://i.scdn.co/image/ab6761610000f178c884df599abc793c116cdf15',
     'width': 160}],
   'name': 'Foo Fighters',
   'popularity': 75,
   'type': 'artist',
   'uri': 'spotify:artist:7jy3rLJdDQY21OgRLCZ9sD'},
  {'external_urls': {'spotify': 'https://open.spotify.com/artist/2d0hyoQ5ynDBnkvAbJKORj'},
   'followers': {'href': 

In [14]:
def get_related_artists(artist_id):          
    related_artists_df = pd.DataFrame(rel_artists_ep_response["artists"])[["id", "name"]]
    return related_artists_df

# Get and display related artists DataFrame
related_artists_df = get_related_artists(artist_id)
print("\nRelated Artists DataFrame:")
related_artists_df

NameError: name 'rel_artists_ep_response' is not defined

In [33]:
parameter = '/top-tracks'
country_market = '?market=ES'   
rel_top_tracks_ep = rel_artists_ep + parameter + country_market    
rel_top_tracks_ep_response = requests.get(rel_top_tracks_ep, headers=header_info).json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

### Aquí empieza a joderse todo

In [16]:
# Crear una columna 'artist_name' en el DataFrame de artistas relacionados
related_artists_df['artist_name'] = related_artists_df['name']

# Inicializar el DataFrame principal de top tracks
all_top_tracks_df = pd.DataFrame(columns=["id", "href", "song_name", "uri", "artist_name"])

# Iterar sobre los artistas relacionados y agregar sus top tracks al DataFrame principal
for index, row in related_artists_df.iterrows():
    related_artist_id = row['id']
    related_artist_name = row['artist_name']
    
    # Obtener los top tracks del artista relacionado y agregarlos al DataFrame principal
    related_artist_top_tracks = get_top_tracks_for_related_artist(related_artist_id, related_artist_name)
    
    # Concatenar el DataFrame del artista relacionado al DataFrame principal
    all_top_tracks_df = pd.concat([all_top_tracks_df, related_artist_top_tracks], ignore_index=True)

# Mostrar el DataFrame de top tracks para todos los artistas relacionados
print("\nTop Tracks for All Related Artists:")

all_top_tracks_df = all_top_tracks_df + artist_top_tracks
all_top_tracks_df






NameError: name 'related_artists_df' is not defined

### 3. Related artists

In [252]:
method = '/related-artists'
# argument = "market = ES"
rel_artists_ep = base_url + resource + artist_id + method 
print(rel_artists_ep)

https://api.spotify.com/v1/artists/0L8ExT028jH3ddEcZwqJJ5/related-artists


In [253]:
rel_artists_ep_response = requests.get(rel_artists_ep, headers=header_info)
rel_artists_ep_response.status_code

200

In [254]:
rel_artists_ep_response.json()

{'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7jy3rLJdDQY21OgRLCZ9sD'},
   'followers': {'href': None, 'total': 11379717},
   'genres': ['alternative metal',
    'alternative rock',
    'modern rock',
    'permanent wave',
    'post-grunge',
    'rock'],
   'href': 'https://api.spotify.com/v1/artists/7jy3rLJdDQY21OgRLCZ9sD',
   'id': '7jy3rLJdDQY21OgRLCZ9sD',
   'images': [{'height': 640,
     'url': 'https://i.scdn.co/image/ab6761610000e5ebc884df599abc793c116cdf15',
     'width': 640},
    {'height': 320,
     'url': 'https://i.scdn.co/image/ab67616100005174c884df599abc793c116cdf15',
     'width': 320},
    {'height': 160,
     'url': 'https://i.scdn.co/image/ab6761610000f178c884df599abc793c116cdf15',
     'width': 160}],
   'name': 'Foo Fighters',
   'popularity': 75,
   'type': 'artist',
   'uri': 'spotify:artist:7jy3rLJdDQY21OgRLCZ9sD'},
  {'external_urls': {'spotify': 'https://open.spotify.com/artist/2d0hyoQ5ynDBnkvAbJKORj'},
   'followers': {'href': 

### 4. Related artist top tracks

In [264]:
# Getting the chosen artist top tracks
def get_top_tracks(artist_id):    
    parameter = '/top-tracks'
    country_market = '?market=ES'   
    top_tracks_ep = artist_ep + parameter + country_market    
    top_tracks_ep_response = requests.get(top_tracks_ep, headers=header_info).json()
    top_tracks_df = pd.DataFrame(top_tracks_ep_response["tracks"])[["id", "href", "name", "uri"]]
    
    return top_tracks_df

artist_top_tracks = get_top_tracks(artist_id)
artist_top_tracks.head()

,id,href,name,uri
0,3d9DChrdc6BOeFsbrZ3Is0,https://api.spotify.com/v1/tracks/3d9DChrdc6BO...,Under the Bridge,spotify:track:3d9DChrdc6BOeFsbrZ3Is0
1,48UPSzbZjgc449aqz8bxox,https://api.spotify.com/v1/tracks/48UPSzbZjgc4...,Californication,spotify:track:48UPSzbZjgc449aqz8bxox
2,3ZOEytgrvLwQaqXreDs2Jx,https://api.spotify.com/v1/tracks/3ZOEytgrvLwQ...,Can't Stop,spotify:track:3ZOEytgrvLwQaqXreDs2Jx
3,1G391cbiT3v3Cywg8T7DM1,https://api.spotify.com/v1/tracks/1G391cbiT3v3...,Scar Tissue,spotify:track:1G391cbiT3v3Cywg8T7DM1
4,64BbK9SFKH2jk86U3dGj2P,https://api.spotify.com/v1/tracks/64BbK9SFKH2j...,Otherside,spotify:track:64BbK9SFKH2jk86U3dGj2P


### 4. Release playlist

In [307]:
# Getting the related-artists
def get_related_artists(artist_id):
    rel_artists_ep = base_url + resource + artist_id + method
    rel_artists_ep_response = requests.get(rel_artists_ep, headers=header_info).json()        
    related_artists_df = pd.DataFrame(rel_artists_ep_response["artists"])[["id", "name"]]
    related_artists_df
    return related_artists_df

related_artists_df = get_related_artists(artist_id)
related_artists_df

,id,name
0,7jy3rLJdDQY21OgRLCZ9sD,Foo Fighters
1,2d0hyoQ5ynDBnkvAbJKORj,Rage Against The Machine
2,2ziB7fzrXBoh1HUPS6sVFn,Audioslave
3,1w5Kfo2jwwIPruYS2UWh56,Pearl Jam
4,40Yq4vzPs9VNUrIBG5Jr2i,The Smashing Pumpkins
5,6olE6TJLqED3rqDCT0FyPh,Nirvana
6,5xUf6j4upBrXZPg6AI4MRK,Soundgarden
7,3YcBF2ttyueytpXtEzn1Za,Incubus
8,5LfGQac0EIXyAN8aUwmNAQ,The Offspring
9,2UazAtjfzqBF0Nho2awK4z,Stone Temple Pilots


In [284]:
def make_playlist(dataframe):
    playlist = pd.DataFrame() 
    for artist_id in dataframe:
        artist_name = related_artists_df[related_artists_df["name"] == artist_id]["id"].values[0]
        artist_top_tracks = get_top_tracks(artist_id)
        artist_top_tracks["artist_name"] = artist_name
        playlist = pd.concat([playlist, artist_top_tracks])
    playlist = playlist.reset_index(drop=True)
    return playlist

In [285]:
playlist = make_playlist(related_artists_df[["id"]])  # Asegúrate de seleccionar solo la columna 'id'
playlist.head(30)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [130]:
# Dummy code

def get_top_tracks(artist_id):
    resource = "artists"
    parameters = f"/{artist_id}/top-tracks?market=ES"
    url = base_url + resource + parameters    
    response = requests.get(url, headers=header_info).json()
    df_tracks = pd.DataFrame(response["tracks"])[["id", "href", "name", "uri"]]
    return df_tracks

### Checkpoint

In [131]:
artist_id = '0L8ExT028jH3ddEcZwqJJ5'

In [132]:
# API call
artist_response = requests.get(url, headers=header_info)
artist_response.status_code

NameError: name 'url' is not defined

In [ ]:
# Create DataFrames
artist_df = pd.DataFrame(dict_artist)
artist_df

In [ ]:
method = '/related-artists'
# argument = "market = ES"
full_endpoint = base_url + resource + parameters + method 
print(full_endpoint)

In [ ]:
related_artists_response = requests.get(full_endpoint, headers = header_info)
related_artists_response.status_code

In [ ]:
related_artists_json = related_artists_response.json()
related_artists_json

In [ ]:
#Ahora voy a sacar los artistas relacionados con sus id.
def get_related_artists(artist_id):
    parameters = f"/{artist_id}/related-artists"
    url = base_url + resource + parameters
    response = related_artists_json
    df_related_artists= pd.DataFrame(response["artists"])[["id", "name"]]
    df_related_artists
    return df_related_artists

In [ ]:
df_related_artists = get_related_artists('0L8ExT028jH3ddEcZwqJJ5')
df_related_artists

### 3. Top-tracks

In [ ]:
# Voy a coger los top_tracks de mi artista seleccionado:

def top_tracks(artist_id):
    resource = "artists"
    parameters = f"/{artist_id}/top-tracks?market=ES"
    url = base_url + resource + parameters
    response = requests.get(url, headers=header_info).json()
    df_tracks = pd.DataFrame(response["tracks"])[["id", "href", "name", "uri"]]
    return df_top_tracks

In [ ]:
def make_playlist(dataframe):
    playlist = pd.DataFrame() 
    for artist_id in dataframe:
        artist_name = df_related_artists[df_related_artists["id"] == artist_id]["name"].values[0]
        top_tracks = top_tracks(artist_id)
        top_tracks["artist_name"] = artist_name
        playlist = pd.concat([playlist, top_tracks])
    playlist = playlist.reset_index(drop=True)
    return playlist

In [ ]:
playlist = make_playlist(df_related_artists["id"])
playlist.head(20)

In [ ]:
'''Para mas tarde'''

# #  Rename columns
# related_artists_df = related_artists_df.rename(columns={'id':'related_id',
#                                                         'name':'related_name',
#                                                         'genres':'related_genres',
#                                                         'popularity': 'popularity',
#                                                         'followers':'related_followers'
#                                                         })
# related_artists_df.head()

In [ ]:
print(artist_df.columns)

### 2. Top-tracks

In [ ]:
# Get top tracks
method = '/top-tracks'
argument = '?market=ES'
full_endpoint = base_url + resource + parameters + method + argument
full_endpoint

In [ ]:
def get_top_tracks(artist_id):
    resource = "artists"
    parameters = f"/{artist_id}/top-tracks?market=ES"
    url = base_url + resource + parameters
    response = requests.get(url, headers=header_info).json()
    df_tracks = pd.DataFrame(response["tracks"])[["id", "href", "name", "uri"]]
    return df_tracks

In [ ]:
get_top_tracks("0L8ExT028jH3ddEcZwqJJ5")

In [ ]:
top_tracks_response = requests.get(full_endpoint, headers = header_info)
top_tracks_response.status_code

In [ ]:
top_tracks_json = top_tracks_response.json()
top_tracks_json.keys()

In [ ]:
dict_top_tracks = dict((key, [top_tracks_json[key]]) for key in ['tracks'] if key in top_tracks_json)

In [ ]:
dict_top_tracks

In [ ]:
# Top Tracks
[top_tracks_json['tracks'][i]['name'] for i in range(len(top_tracks_json['tracks']))]

In [ ]:
# Create DataFrames
top_tracks_df = pd.DataFrame(top_tracks_json['tracks'])[['id','name']]
top_tracks_df['tracks'] = artist_id
top_tracks_df

In [ ]:
print(artist_df.columns)
print(top_tracks_df.columns)

### 3. Related artists

In [ ]:
# Get Related artists



In [ ]:
related_artists_response = requests.get(full_endpoint, headers = header_info)
related_artists_response.status_code

In [ ]:
related_artists_json = related_artists_response.json()
related_artists_json

In [ ]:
# Related Artists
[related_artists_json['artists'][i]['name'] for i in range(len(related_artists_json['artists']))]

In [ ]:
# Create DataFrames
related_artists_df = pd.DataFrame(related_artists_json['artists'])[['id','name','genres','popularity','followers']]
related_artists_df

In [ ]:
# #  Rename columns
# related_artists_df = related_artists_df.rename(columns={'id':'related_id',
#                                                         'name':'related_name',
#                                                         'genres':'related_genres',
#                                                         'popularity': 'popularity',
#                                                         'followers':'related_followers'
#                                                         })
# related_artists_df.head()

In [ ]:
def make_playlist(dataframe):
    playlist = pd.DataFrame() 
    for artist_df in dataframe:
        artist_name = related_artists_df[related_artists_df["id"] == artist_df]["name"].values[0]
        top_tracks = top_tracks_df["id"]
        top_tracks["name"] = artist_name
        playlist = pd.concat([playlist, top_tracks])
    playlist = playlist.reset_index(drop=True)
    return playlist
make_playlist(related_artists_df)

In [ ]:
# Add a column with selected id artist
related_artists_df['artist_id'] = artist_id
related_artists_df.head()

In [ ]:
print(related_artists_df.columns)
print(artist_df.columns)
print(top_tracks_df.columns)

In [ ]:
# Join artist and related artist to merge info

# For loop de mi artista con los top tracks, teniendo el id_artist de los related
# Crear una lista de los ids los related

# Los 20 top tracks de los 10 arti


In [ ]:
related_artists_df.head()

# Checkpoint

---

### Bonus track!!!

You can publish your own Playlist with [`/playlists`](https://developer.spotify.com/documentation/web-api/reference/#/operations/create-playlist) and [`/tracks`](https://developer.spotify.com/documentation/web-api/reference/#/operations/add-tracks-to-playlist).

![I quit](https://media.giphy.com/media/gui67fZ3xIneM/giphy.gif)

In [ ]:
# Bonus
import json
import string
import random
import base64



---

You can always try with the [wrapper](https://github.com/plamere/spotipy)!!!

![I quit](https://media.giphy.com/media/3oFzlXPvXYZ4q8VQOs/giphy.gif)